# Analyse d'impact d'événements sur les marchés financiers

On considère 5 évènements disruptifs majeurs :

1. Catastrophe naturelle :
Incendies en Californie - 8 novembre 2018 (début)

2. Evenement sectoriel :
Scandale des opioïdes (Purdue Pharma & Big Pharma) - 23 octobre 2017 (premier procès d'État majeur contre les fabricants d’opioïdes)

3. Evenement entreprise :
Apple : Publication résultats Q4 catastrophiques - 2 janvier 2019

4. Evenement mondial :
Invasion de l’Ukraine par la Russie - 24 février 2022 (début officiel à l’aube)

5. Evenemment non américano centré :
Brexit - 23 Juin 2016

In [ ]:
# Potentiellement inutile si déjà fait plus tôt dans le notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta


def load_and_prepare_data(csv_path):
    """
    Charge les données et ajoute les informations sectorielles du S&P 500.
    
    Parameters
    ----------
    csv_path : str
        Chemin vers le fichier CSV contenant les données financières
        
    Returns
    -------
    pd.DataFrame
        DataFrame avec les colonnes enrichies (Date, Ticker, Sector, etc.)
    """
    # Chargement des données principales
    df = pd.read_csv(csv_path)
    
    # Récupération de la liste S&P 500 depuis Wikipedia
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    
    tables = pd.read_html(resp.text, header=0)
    sp500 = tables[0]
    
    # Nettoyage et préparation des données S&P 500
    sp500 = sp500.rename(columns={"Symbol": "Ticker", "GICS Sector": "Sector"})
    sp500['Ticker'] = sp500['Ticker'].str.replace('.', '-', regex=False)
    
    # Tri et merge
    df = df.sort_values(by=['Ticker', 'Date'])
    df = df.merge(sp500[['Ticker', 'Sector']], on='Ticker', how='left')
    
    # Conversion de la date
    df['Date'] = pd.to_datetime(df['Date'])
    
    return df

csv_path = "dataset.csv"   
# Chargement et préparation des données
df = load_and_prepare_data(csv_path)

In [ ]:
def extract_event_window(df, event_date, window_days=15):
    """
    Extrait une fenêtre temporelle autour d'un événement.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame contenant les données financières
    event_date : str or pd.Timestamp
        Date de l'événement (format 'YYYY-MM-DD')
    window_days : int, optional
        Nombre de jours avant et après l'événement (default: 15)
        
    Returns
    -------
    pd.DataFrame
        DataFrame filtré sur la fenêtre [-window_days, +window_days]
    """
    event_date = pd.to_datetime(event_date)
    start_date = event_date - pd.Timedelta(days=window_days)
    end_date = event_date + pd.Timedelta(days=window_days)
    
    df_event = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].copy()
    
    return df_event


def plot_metric_comparison(df_subsets, labels, metric, event_date, 
                           event_name, ylabel, title_prefix, figsize=(16, 6)):
    """
    Crée un graphique comparatif de métriques pour plusieurs sous-ensembles de données.
    
    Parameters
    ----------
    df_subsets : list of pd.DataFrame
        Liste des DataFrames à comparer
    labels : list of str
        Labels pour chaque sous-ensemble
    metric : str
        Nom de la colonne métrique à analyser (ex: 'Return', 'Volatility')
    event_date : pd.Timestamp
        Date de l'événement
    event_name : str
        Nom de l'événement pour le titre
    ylabel : str
        Label de l'axe Y
    title_prefix : str
        Préfixe du titre général
    figsize : tuple, optional
        Taille de la figure (default: (16, 6))
        
    Returns
    -------
    matplotlib.figure.Figure
        Figure matplotlib créée
    """
    n_plots = len(df_subsets)
    fig, axes = plt.subplots(1, n_plots, figsize=(figsize[0] * n_plots / 2, figsize[1]))
    
    # Gérer le cas d'un seul subplot
    if n_plots == 1:
        axes = [axes]
    
    # Stocker toutes les valeurs pour synchroniser les échelles
    all_values = []
    
    for df_subset, label, ax in zip(df_subsets, labels, axes):
        # Calculer la moyenne quotidienne de la métrique
        daily_avg = df_subset.groupby('Date')[metric].mean() * 100
        all_values.extend(daily_avg.values)
        
        # Créer l'index de jours relatifs
        dates = daily_avg.index.sort_values()
        days_relative = [(d - event_date).days for d in dates]
        
        # Tracer la courbe
        ax.plot(days_relative, daily_avg.loc[dates].values,
                linewidth=2.5, color='navy', marker='o', markersize=4, alpha=0.8)
        
        # Ligne verticale pour l'événement
        ax.axvline(x=0, color='red', linestyle='--', linewidth=2.5,
                   label='Événement', alpha=0.8)
        
        # Ligne horizontale à 0
        ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.8, alpha=0.5)
        
        # Zones avant/après colorées
        ax.axvspan(-15, 0, alpha=0.1, color='blue', label='Avant')
        ax.axvspan(0, 15, alpha=0.1, color='red', label='Après')
        
        # Labels et titre
        ax.set_xlabel('Jours relatifs à l\'événement', fontsize=12, fontweight='bold')
        ax.set_ylabel(ylabel, fontsize=12, fontweight='bold')
        ax.set_title(f'{label}\n({len(df_subset["Ticker"].unique())} tickers)',
                     fontsize=13, fontweight='bold')
        
        # Grille et légende
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.legend(loc='best', fontsize=10)
        ax.set_xlim(-15, 15)
        
        # Statistiques
        mean_before = df_subset[df_subset['Date'] < event_date][metric].mean() * 100
        mean_after = df_subset[df_subset['Date'] > event_date][metric].mean() * 100
        
        stats_text = f'Moy. avant: {mean_before:.3f}%\nMoy. après: {mean_after:.3f}%'
        ax.text(0.02, 0.98, stats_text, transform=ax.transAxes,
                fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Synchroniser les échelles Y
    y_min = min(all_values)
    y_max = max(all_values)
    y_margin = (y_max - y_min) * 0.1
    
    for ax in axes:
        ax.set_ylim(y_min - y_margin, y_max + y_margin)
    
    # Titre général
    fig.suptitle(f'{title_prefix} [-15j, +15j] - {event_name}',
                 fontsize=16, fontweight='bold', y=0.98)
    
    plt.tight_layout()
    
    return fig

def calculate_sector_delta_volume(df, event_date, window_days=15):
    """
    Calcule le delta de volume (avant/après) par secteur pour un événement donné.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame complet avec toutes les données
    event_date : str or pd.Timestamp
        Date de l'événement
    window_days : int, optional
        Nombre de jours avant et après l'événement (default: 15)
        
    Returns
    -------
    list of dict
        Liste de dictionnaires contenant les statistiques par secteur
    """
    event_date = pd.to_datetime(event_date)
    df_event = extract_event_window(df, event_date, window_days)
    
    df_before = df_event[df_event['Date'] < event_date]
    df_after = df_event[df_event['Date'] > event_date]
    
    sectors = df_event['Sector'].unique()
    results = []
    
    for sector in sectors:
        sector_before = df_before[df_before['Sector'] == sector]
        sector_after = df_after[df_after['Sector'] == sector]
        
        zscore_before = sector_before['Volume_z'].mean()
        zscore_after = sector_after['Volume_z'].mean()
        delta_volume = zscore_after - zscore_before
        
        n_tickers = df_event[df_event['Sector'] == sector]['Ticker'].nunique()
        
        results.append({
            'Secteur': sector,
            'N_Tickers': n_tickers,
            'Volume_Avant': zscore_before,
            'Volume_Apres': zscore_after,
            'Delta_Volume': delta_volume,
        })
    
    return results

def plot_sector_delta_volume(results, event_name, figsize=(20, 18)):
    """
    Crée un graphique en barres horizontales du delta de volume par secteur.
    
    Parameters
    ----------
    results : list of dict
        Résultats calculés par calculate_sector_delta_volume()
    event_name : str
        Nom de l'événement pour le titre
    figsize : tuple, optional
        Taille de la figure (default: (20, 18))
        
    Returns
    -------
    matplotlib.figure.Figure
        Figure matplotlib créée
    """
    # Palette de couleurs par secteur
    sector_colors = {
        'Health Care': '#e74c3c',
        'Information Technology': '#3498db',
        'Financials': '#2ecc71',
        'Consumer Staples': '#f39c12',
        'Industrials': '#9b59b6',
        'Utilities': '#1abc9c',
        'Materials': '#e67e22',
        'Real Estate': '#34495e',
        'Consumer Discretionary': '#16a085',
        'Energy': '#d35400',
        'Communication Services': '#8e44ad'
    }
    
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    
    # Préparation des données
    df_results = pd.DataFrame(results).sort_values('Delta_Volume', ascending=True)
    y_pos = np.arange(len(df_results))
    colors = [sector_colors.get(s, '#95a5a6') for s in df_results['Secteur']]
    
    # Création des barres
    bars = ax.barh(y_pos, df_results['Delta_Volume'], 
                   color=colors, alpha=0.8, edgecolor='black', linewidth=1.2)
    
    # Ajout des valeurs sur les barres
    for i, (idx, row) in enumerate(df_results.iterrows()):
        value = row['Delta_Volume']
        ax.text(value + (0.1 if value > 0 else -0.1), i, f"{value:.2f}", 
                va='center', ha='left' if value > 0 else 'right',
                fontsize=9, fontweight='bold')
    
    # Configuration des axes
    ax.set_yticks(y_pos)
    ax.set_yticklabels([f"{s} ({n})" for s, n in 
                        zip(df_results['Secteur'], df_results['N_Tickers'])],
                        fontsize=10, fontweight='bold')
    ax.set_xlabel('Δ Volume (Z-score)', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    ax.axvline(x=0, color='black', linewidth=1.5)
    
    # Titre
    fig.suptitle(f'Comparaison d\'Impact par Secteur: {event_name}\nDelta volume',
                fontsize=18, fontweight='bold', y=0.995)
    
    plt.tight_layout()
    
    return fig

I. Incendies en Californie

In [ ]:
def analyze_california_fire(df):
    """
    Analyse l'impact de l'incendie de Californie (2018-11-08) sur PG&E.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame complet avec toutes les données
    """
    event_date = pd.to_datetime('2018-11-08')
    df_event = extract_event_window(df, event_date)
    
    # Sous-ensembles : PG&E vs Tous les tickers
    df_pcg = df_event[df_event['Ticker'] == 'PCG'].copy()
    df_all = df_event.copy()
    
    # Graphique 1 : Rendements quotidiens
    plot_metric_comparison(
        df_subsets=[df_pcg, df_all],
        labels=['Tickers PG&E', 'Tous Tickers'],
        metric='Return',
        event_date=event_date,
        event_name='Incendie de Californie',
        ylabel='Rendement quotidien moyen (%)',
        title_prefix='Rendements Quotidiens Moyens'
    )
    plt.show()
    
    # Graphique 2 : Volatilité
    plot_metric_comparison(
        df_subsets=[df_pcg, df_all],
        labels=['Tickers PG&E', 'Tous Tickers'],
        metric='Volatility',
        event_date=event_date,
        event_name='Incendie de Californie',
        ylabel='Volatilité moyenne (%)',
        title_prefix='Volatilité moyenne'
    )
    plt.show()


analyze_california_fire(df)

Le graphique de rendements montre que l’incendie de Californie a un impact très marqué sur PG&E, dont les rendements moyens chutent nettement après l’événement, tandis que le reste du marché reste pratiquement inchangé. Cette réaction différenciée s’explique par le fait que PG&E est directement mise en cause dans l’origine de l’incendie, ce qui fait exploser son risque juridique et financier. Les investisseurs anticipent des amendes colossales, des indemnisations massives et même un risque de faillite, ce qui entraîne une réévaluation brutale de l’action et donc une baisse persistante de ses rendements. À l’inverse, les autres entreprises n’ont aucun lien économique avec l’événement, ne subissent aucune exposition directe, on ne mesure donc pas d'impact direct sur leur rendement.

Le graphique de volatilité montre une explosion très nette de la variabilité des rendements pour PG&E immédiatement après l’incendie, alors que la volatilité des autres entreprises reste globalement stable. Cette hausse extrême de volatilité traduit l’incertitude massive à laquelle l’entreprise fait soudain face : les investisseurs ne savent pas encore si PG&E devra payer des dizaines de milliards de dollars, si elle sera poursuivie au civil, si elle pourra lever suffisamment de capitaux ou même survivre à l’événement, ce qui provoque des fluctuations intenses des anticipations et donc du prix. À l’inverse, les autres entreprises ne subissent aucun changement de risque fondamental, ce qui explique leur volatilité pratiquement inchangée.

II. Scandale des opioïdes

In [ ]:
def analyze_opioid_scandal(df):
    """
    Analyse l'impact du scandale des opioïdes (2017-10-23) sur le secteur Health Care.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame complet avec toutes les données
    """
    event_date = pd.to_datetime('2017-10-23')
    df_event = extract_event_window(df, event_date)
    
    # Sous-ensembles : Health Care vs Non Health Care
    df_hc = df_event[df_event['Sector'] == 'Health Care'].copy()
    df_other = df_event[df_event['Sector'] != 'Health Care'].copy()
    
    # Graphique : Volatilité
    plot_metric_comparison(
        df_subsets=[df_hc, df_other],
        labels=['Tickers Health Care', 'Tickers non Health Care'],
        metric='Volatility',
        event_date=event_date,
        event_name='Scandale des opioïdes',
        ylabel='Volatilité moyenne (%)',
        title_prefix='Volatilité moyenne'
    )
    plt.show()

analyze_opioid_scandal(df)

Le graphique montre que, autour du scandale des opioïdes, la volatilité des entreprises du secteur de la santé (healthcare) augmente sensiblement, tandis que celle des entreprises hors santé reste plus stable. Cette divergence reflète la manière dont l’événement touche de façon asymétrique les acteurs du marché : le scandale entraîne une incertitude juridique, politique et financière importante pour les laboratoires pharmaceutiques et les distributeurs impliqués dans la crise des opioïdes (risques de poursuites massives, amendes fédérales, régulations renforcées et réputation durablement dégradée) ce qui accroît brutalement l’incertitude perçue par les investisseurs et donc la volatilité de leurs actions. À l’inverse, les entreprises non liées à la santé ne sont pas exposées à ces risques spécifiques et ne voient donc pas leur structure de risque évoluer. La hausse de volatilité dans le groupe healthcare traduit ainsi l’impact direct du choc d’information, tandis que la stabilité du groupe non healthcare confirme que l’événement est sectoriel plutôt que systémique.

III. Publication résultats Q4 catastrophiques d'Apple

In [ ]:
def analyze_apple_results(df):
    """
    Analyse l'impact des résultats Apple (2019-01-02) sur Apple et le secteur IT.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame complet avec toutes les données
    """
    event_date = pd.to_datetime('2019-01-02')
    df_event = extract_event_window(df, event_date)
    
    # Sous-ensembles : Apple, IT (sans Apple), Non IT
    df_apl = df_event[df_event['Ticker'] == 'AAPL'].copy()
    df_it = df_event[(df_event['Sector'] == 'Information Technology') & 
                     (df_event['Ticker'] != 'AAPL')].copy()
    df_other = df_event[df_event['Sector'] != 'Information Technology'].copy()
    
    # Graphique 1 : Volatilité
    plot_metric_comparison(
        df_subsets=[df_apl, df_it, df_other],
        labels=['Tickers Apple', 'Tickers IT', 'Tickers non IT'],
        metric='Volatility',
        event_date=event_date,
        event_name='Résultats Apple',
        ylabel='Volatilité moyenne (%)',
        title_prefix='Volatilité moyenne',
        figsize=(24, 6)
    )
    plt.show()
    
    # Graphique 2 : Volume
    plot_metric_comparison(
        df_subsets=[df_apl, df_it, df_other],
        labels=['Tickers Apple', 'Tickers IT', 'Tickers non IT'],
        metric='Volume_z',
        event_date=event_date,
        event_name='Résultats Apple',
        ylabel='Volume moyen (%)',
        title_prefix='Volume Moyens',
        figsize=(24, 6)
    )
    plt.show()

    analyze_apple_results(df)

L’ensemble des graphiques montre que l’événement affectant Apple déclenche une réaction de marché forte et spécifique à l’entreprise, alors que les autres acteurs, qu’ils appartiennent au secteur technologique ou non, restent largement insensibles. D'abord, la volatilité d’Apple augmente nettement, révélant une montée de l’incertitude liée au choc d’information, alors que la volatilité des groupes IT et non-IT reste stable, confirmant l’absence de risque systémique. Ensuite, les volumes de transaction explosent pour Apple, signe d’un repositionnement massif des investisseurs (ventes paniques et prises de positions opportunistes) alors que les autres entreprises ne connaissent pas de changement notable dans leur liquidité.

IV Invasion de l’Ukraine par la Russie

In [ ]:
def analyze_ukraine_war(df):
    """
    Analyse l'impact de la guerre en Ukraine (2022-02-24) par secteur.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame complet avec toutes les données
    """
    event_date = pd.to_datetime('2022-02-24')
    results = calculate_sector_delta_volume(df, event_date)
    plot_sector_delta_volume(results, 'Guerre en Ukraine')
    plt.show()

analyze_ukraine_war(df)

Le graphique montre clairement que le déclenchement de la guerre en Ukraine a provoqué un choc sectoriel asymétrique, avec des impacts très différents selon les industries : la guerre déclenche une envolée particulièrement marquée des volumes dans les secteurs les plus exposés aux répercussions immédiates du conflit (notamment l’Énergie et les Matériaux) traduisant une réallocation rapide du capital vers les « gagnants » anticipés de la crise, tandis que des secteurs moins concernés présentent des mouvements plus modérés. La guerre agit comme un choc géopolitique majeur, provoquant une réaction violente mais différenciée selon l’exposition sectorielle aux matières premières, au risque global et aux perspectives macroéconomiques.

V. Brexit

In [ ]:
def analyze_brexit(df):
    """
    Analyse l'impact du Brexit (2016-06-23) par secteur.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame complet avec toutes les données
    """
    event_date = pd.to_datetime('2016-06-23')
    results = calculate_sector_delta_volume(df, event_date)
    plot_sector_delta_volume(results, 'Brexit')
    plt.show()

analyze_brexit(df)

Le Brexit a provoqué un impact sectoriel sur le volume des actions du S&P 500 principalement axé sur l'incertitude réglementaire et les perspectives économiques régionales, comme en témoignent les secteurs des Services Publics (Utilities) et de la Consommation Discrétionnaire qui ont été les plus affectés. Contrairement à la guerre en Ukraine, dont l'effet dominant s'est concentré sur les matières premières et l'Énergie en raison du choc d'offre mondial, l'impact du Brexit a été un choc plus largement distribué, touchant le secteur de l'Énergie de manière marginale.